In [13]:
!pip install -U huggingface_hub
!pip install transformers

In [42]:
import transformers
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [36]:
import os
import re
import pandas as pd
count = 0
for root, folders, filenames in os.walk('/kaggle/input'):
    print(root, folders)

/kaggle/input ['finetuned-model', 'amazon-fine-food-reviews']
/kaggle/input/finetuned-model ['model']
/kaggle/input/finetuned-model/model []
/kaggle/input/amazon-fine-food-reviews []


In [29]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('/kaggle/input/finetuned-model/model/')

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [34]:
def cleaning(s):
    s = str(s)
    s = re.sub('\s\W',' ',s)
    s = re.sub('\W,\s',' ',s)
    s = re.sub("\d+", "", s)
    s = re.sub('\s+',' ',s)
    s = re.sub('[!@#$_]', '', s)
    s = s.replace("co","")
    s = s.replace("https","")
    s = s.replace("[\w*"," ")
    return s + '\nTL;DR:\n'

In [41]:
df = pd.read_csv("/kaggle/input/amazon-fine-food-reviews/Reviews.csv", encoding="ISO-8859-1") 
df = df.dropna(subset = ['Text', 'Summary'])
df['Text'] = df['Text'].apply(lambda x: cleaning(x))
subdf = df.head(30000)
subdf


NameError: name 'train_test_split' is not defined

In [43]:
training_data, testing_data = train_test_split(subdf, test_size = 0.25, random_state = 42)

In [38]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=015c4e2e7e6d61ad27a3d492ded22d6443b7b2249ce4746f13350a470da57b36
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [48]:
import pandas as pd
from rouge_score import rouge_scorer

import torch
if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token is None: tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def generate_summary(text, model, tokenizer, device):
    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    summary_ids = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_new_tokens=128, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def Results(result, lst):
    res = {}
    for item in lst:
        if item[0].split('_')[1] == 'precision':
            res[item[0]] = result[item[1]].precision
        elif item[0].split('_')[1] == 'recall':
            res[item[0]] = result[item[1]].recall
        else:
            res[item[0]] = result[item[1]].fmeasure
    return res

def evaluate_rouge(data, model, tokenizer, device):
    scoring = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    results = []
    for index, row in data.iterrows():
        text = row['Text']
        summary = row['Summary']
        
        generated_summary = generate_summary(text, model, tokenizer, device)
#         print("Text:", text)
#         print("Summary:", summary)
#         print("GenSummary:", generated_summary[len(text):])
        result = scoring.score(summary, generated_summary[len(text):])
        
        localresult = Results(result, [['rouge1_precision', 'rouge1'], ['rouge1_recall', 'rouge1'], ['rouge1_fmeasure', 'rouge1'], ['rouge2_precision', 'rouge2'], ['rouge2_recall', 'rouge2'], ['rouge2_fmeasure', 'rouge2'], ['rougeL_precision', 'rougeL'], ['rougeL_recall', 'rougeL'], ['rougeL_fmeasure', 'rougeL']])
#         print(localresult)
        results.append(localresult)

    return pd.DataFrame(results)

model = model.to('cuda')

final_df = evaluate_rouge(testing_data, model, tokenizer, 'cuda')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [50]:
final_df.to_csv('final_metric.csv', index = True)

In [52]:
!zip "final_metric.zip" "final_metric.csv"

  adding: final_metric.csv (deflated 91%)
